# Using ADAM-API to access UERRA regional reanalysis

- you need to get an account to https://reliance.adamplatform.eu/ (use ORCID to authenticate) and key your ADAM API key
- make sure you save your ADAM API key in a file `$HOME/adam-key`

In [ ]:
!pip install adamapi

In [ ]:
import os
import glob
import pathlib
import zipfile
import adamapi as adam
import xarray as xr
from datetime import datetime
import matplotlib
import cartopy.crs as ccrs
from cmcrameri import cm

In [ ]:
adam_key = open(os.path.join(os.environ["HOME"], "adam-key")).read().rstrip()

In [ ]:
a = adam.Auth()

a.setKey(adam_key)
a.setAdamCore("https://reliance.adamplatform.eu")
a.authorize()

## Discover UERRA datasets 
- This step is useful to get the dataset identifier (unique for a given datacube)

In [ ]:
def discoverDasasets(a, search_name):
    datasets = adam.Datasets(a)
    catalogue = datasets.getDatasets()
    # Extracting the size of the catalogue
    total = catalogue["properties"]["totalResults"]
    items = catalogue["properties"]["itemsPerPage"]
    pages = total // items

    print("----------------------------------------------------------------------")
    print("\033[1m" + "List of available datasets:")
    print("\033[0m")

    # Extracting the list of datasets across the whole catalogue
    for i in range(0, pages):
        page = datasets.getDatasets(page=i)
        for element in page["content"]:
            if search_name in element["title"]:
                print(
                    element["title"]
                    + "\033[1m"
                    + " --> datasetId "
                    + "\033[0m"
                    + "= "
                    + element["datasetId"]
                )
    return datasets

In [ ]:
datasets = discoverDasasets(a, "UERRA")

## Get metadata from Snow density

In [ ]:
datasetID = "71100:UERRA_SNOW_DENSITY"

print("\033[1;34m" + "Metadata of " + datasetID + ":")
print("\033[0;0m")

paged = datasets.getDatasets(datasetID)
for i in paged.items():
    print("\033[1m" + str(i[0]) + "\033[0m" + ": " + str(i[1]))

## Discover and select products from a dataset
- for a given time range and spatial coverage

### Get data over the Nordics countries
- The geometry field is extracted from a GeoJSON object , retrieving the value of the "feature" element.

#### Search data
- only print the first 10 products
- UERRA reanalysis are provided 4 times a day (00, 06, 12, 18 UTC)

In [ ]:
!pip install geojson_rewind

In [ ]:
from adamapi import Search
from geojson_rewind import rewind
import json

The GeoJson object needs to be rearranged according to the counterclockwise winding order.This operation is executed in the next few lines to obtain a geometry that meets the requirements of the method. Geom_1 is the final result to be used in the discovery operation.
- you can go to https://geojson.io/ to draw an area of interest (save the produced geojson to a file)

In [ ]:
with open("nordics.geojson") as f:
    geom_dict = json.load(f)
output = rewind(geom_dict)
geom_1 = str(geom_dict["features"][0]["geometry"])

In [ ]:
geom_1

In [ ]:
start_date = "2015-01-01T00:00:00Z"
end_date = "2015-12-31T00:00:00Z"

In [ ]:
search = Search(a)
results = search.getProducts(
    datasetID, geometry=geom_1, startDate=start_date, endDate=end_date
)

# Printing the results

print("\033[1m" + "List of available products:")
print("\033[0m")
count = 1
for i in results["content"]:

    print("\033[1;31;1m" + "#" + str(count))
    print("\033[0m")
    for k in i.items():
        print(str(k[0]) + ": " + str(k[1]))
    count = count + 1
    print("------------------------------------")

## Get data
- be aware that you alwasy get daily average from ADAM-API 

In [ ]:
def getZipData(auth, dataset_info):
    if not (
        pathlib.Path(pathlib.Path(dataset_info["outputFname"]).stem).exists()
        or pathlib.Path(dataset_info["outputFname"]).exists()
    ):
        data = adam.GetData(auth)
        image = data.getData(
            datasetId=dataset_info["datasetID"],
            startDate=dataset_info["startDate"],
            endDate=dataset_info["endDate"],
            geometry=dataset_info["geometry"],
            outputFname=dataset_info["outputFname"],
        )
        print(image)

In [ ]:
%%time

output_file = "./UERRA_SNOW_DENSITY_SWE_ADAMAPI_" + start_date + "-" + end_date + ".zip"

datasetInfo = {
    "datasetID": datasetID,
    "startDate": start_date,
    "endDate": end_date,
    "geometry": geom_1,
    "outputFname": output_file,
}
getZipData(a, datasetInfo)

## Data analysis and Visualization

### Unzip data 

In [ ]:
def unzipData(filename):
    with zipfile.ZipFile(filename, "r") as zip_ref:
        zip_ref.extractall(path=pathlib.Path(filename).stem)

In [ ]:
if not pathlib.Path(pathlib.Path(output_file).stem).exists():
    unzipData(output_file)

## Read data in xarray

In [ ]:
def paths_to_datetimeindex(paths):
    return [
        datetime.strptime(date.split("_")[-1].split(".")[0], "%Y-%m-%dt%f")
        for date in paths
    ]

In [ ]:
def getData(dirtif, varname):
    geotiff_list = glob.glob(dirtif)
    # Create variable used for time axis
    time_var = xr.Variable("time", paths_to_datetimeindex(geotiff_list))
    # Load in and concatenate all individual GeoTIFFs
    geotiffs_da = xr.concat(
        [xr.open_rasterio(i, parse_coordinates=True) for i in geotiff_list],
        dim=time_var,
    )
    # Covert our xarray.DataArray into a xarray.Dataset
    geotiffs_da = geotiffs_da.to_dataset("band")
    # Rename the dimensions to make it CF-convention compliant
    geotiffs_da = geotiffs_da.rename_dims({"y": "latitude", "x": "longitude"})
    # Rename the variable to a more useful name
    geotiffs_da = geotiffs_da.rename_vars(
        {1: varname, "y": "latitude", "x": "longitude"}
    )
    # set attribute to variable
    geotiffs_da[varname].attrs = {
        "units": geotiffs_da.attrs[varname + "#units"],
        "long_name": geotiffs_da.attrs[varname + "#long_name"],
    }
    return geotiffs_da

In [ ]:
path_files = os.path.join(pathlib.Path(output_file).stem, "*.tif")
geotiff_ds = getData(path_files, "rsn")

In [ ]:
geotiff_ds

## Analysis
- generate seasonal average

In [ ]:
geotiff_dm = geotiff_ds.groupby("time.season").mean(
    "time", keep_attrs=True, skipna=True
)

In [ ]:
geotiff_dm

In [ ]:
proj_plot = ccrs.Mercator()

p = geotiff_dm["rsn"].plot(
    x="longitude",
    y="latitude",
    transform=ccrs.PlateCarree(),
    aspect=geotiff_dm.dims["longitude"]
    / geotiff_dm.dims["latitude"],  # for a sensible figsize
    subplot_kws={"projection": proj_plot},
    col="season",
    col_wrap=2,
    robust=True,
    cmap=cm.devon_r,
)
# We have to set the map's options on all four axes
for ax, i in zip(p.axes.flat, geotiff_dm.season.values):
    ax.coastlines()
    ax.set_title("Season " + i, fontsize=18)

fig = matplotlib.pyplot.gcf()
fig.set_size_inches(15.0, 15.0)
fig.savefig("UERRA_rsn.png", dpi=100)